# Tutorial: Counterfactual explanations for scorecard with continuous target

This tutorial shows how to generate counterfactual explanations on scorecard models with continuous target. The dataset for this tutorial is https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html.

In [1]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import HuberRegressor

from optbinning import BinningProcess
from optbinning import Scorecard
from optbinning.scorecard import Counterfactual

Load the dataset

In [2]:
data = fetch_california_housing()

target = "target"
variable_names = data.feature_names
X = pd.DataFrame(data.data, columns=variable_names)
y = data.target

#### Scorecard model

First, we develop a scorecard following the steps presented in previous tutorials.

In [3]:
binning_process = BinningProcess(variable_names)

estimator = HuberRegressor(max_iter=200)

scorecard = Scorecard(binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 0, "max": 100},
                      reverse_scorecard=True,verbose=True)

scorecard.fit(X, y)

2022-04-10 13:47:17,806 | INFO : Scorecard building process started.
2022-04-10 13:47:17,808 | INFO : Options: check parameters.
2022-04-10 13:47:17,810 | INFO : Dataset: continuous target.
2022-04-10 13:47:17,811 | INFO : Binning process started.
2022-04-10 13:47:19,943 | INFO : Binning process terminated. Time: 2.1308s
2022-04-10 13:47:19,944 | INFO : Fitting estimator.
2022-04-10 13:47:20,423 | INFO : Fitting terminated. Time 0.4788s
2022-04-10 13:47:20,428 | INFO : Scorecard table building started.
2022-04-10 13:47:20,591 | INFO : Scorecard table terminated. Time: 0.1632s
2022-04-10 13:47:20,592 | INFO : Scorecard building process terminated. Time: 2.7853s


Scorecard(binning_process=BinningProcess(variable_names=['MedInc', 'HouseAge',
                                                         'AveRooms',
                                                         'AveBedrms',
                                                         'Population',
                                                         'AveOccup', 'Latitude',
                                                         'Longitude']),
          estimator=HuberRegressor(max_iter=200), reverse_scorecard=True,
          scaling_method='min_max',
          scaling_method_params={'max': 100, 'min': 0}, verbose=True)

#### Generating counterfactual explanations

As an input data point or query, we select the first sample. Note that a query must be either a dictionary of a pandas DataFrame.

In [4]:
query = X.iloc[0, :].to_frame().T

In [5]:
query

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23


In [6]:
scorecard.predict(query)

array([4.29854244])

The predicted outcome (house value) for this query (house) is 4.3. We want to generate counterfactual explanations to find out how to increase the house value to at least 4.5.

In [7]:
cf = Counterfactual(scorecard=scorecard, verbose=True)

In [8]:
cf.fit(X)

2022-04-10 13:47:20,714 | INFO : Counterfactual fit started.
2022-04-10 13:47:20,715 | INFO : Options: check parameters.
2022-04-10 13:47:20,717 | INFO : Compute optimization problem data.
2022-04-10 13:47:20,778 | INFO : Counterfactual fit terminated. Time: 0.0648s


Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['MedInc',
                                                                                  'HouseAge',
                                                                                  'AveRooms',
                                                                                  'AveBedrms',
                                                                                  'Population',
                                                                                  'AveOccup',
                                                                                  'Latitude',
                                                                                  'Longitude']),
                                   estimator=HuberRegressor(max_iter=200),
                                   reverse_scorecard=True,
                                   scaling_method='min_max',
                                   scaling_method

In [9]:
cf.generate(query=query, y=4.5, outcome_type="continuous", n_cf=1,
            max_changes=3, hard_constraints=["min_outcome"])

2022-04-10 13:47:20,830 | INFO : Counterfactual generation started.
2022-04-10 13:47:20,841 | INFO : Options: check parameters.
2022-04-10 13:47:20,867 | INFO : Options: check objectives and constraints.
2022-04-10 13:47:20,868 | INFO : Optimizer started.
2022-04-10 13:47:20,870 | INFO : Optimizer: build model...
2022-04-10 13:47:20,933 | INFO : Optimizer: solve...
2022-04-10 13:47:21,023 | INFO : Optimizer terminated. Time: 0.1530s
2022-04-10 13:47:21,024 | INFO : Post-processing started.
2022-04-10 13:47:21,040 | INFO : Post-processing terminated. Time: 0.0156s
2022-04-10 13:47:21,042 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 0.2120s


Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['MedInc',
                                                                                  'HouseAge',
                                                                                  'AveRooms',
                                                                                  'AveBedrms',
                                                                                  'Population',
                                                                                  'AveOccup',
                                                                                  'Latitude',
                                                                                  'Longitude']),
                                   estimator=HuberRegressor(max_iter=200),
                                   reverse_scorecard=True,
                                   scaling_method='min_max',
                                   scaling_method

In [10]:
cf.information()

optbinning (Version 0.14.0)
Copyright (c) 2019-2022 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   42
    Number of constraints                120
    Objective value                   7.7965
    Best objective bound              7.7965

  Objectives
    proximity                         0.9862
    closeness                         6.8103

  Timing
    Total time                          0.23 sec
    Fit                                 0.06 sec   ( 27.77%)
    Solver                              0.15 sec   ( 65.57%)
    Post-processing                     0.02 sec   ( 10.16%)



The generate counterfactual suggest increasing the block population, reduce the average house occupancy and change the house block longitude. None of them seems doable.

In [11]:
cf.display(show_only_changes=True, show_outcome=True)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,-,-,-,"[1.05, 1.07)","[986.50, 1426.50)",-,"[34.10, 34.18)",-,4.522533


Now, let's generate several counterfactuals aiming to limit the house value to 4.0.

In [12]:
cf.generate(query=query, y=4.0, outcome_type="continuous", n_cf=3,
            max_changes=3,
            hard_constraints=["diversity_features", "max_outcome"],
            time_limit=30
           ).display(show_only_changes=True, show_outcome=True)

2022-04-10 13:47:21,118 | INFO : Counterfactual generation started.
2022-04-10 13:47:21,120 | INFO : Options: check parameters.
2022-04-10 13:47:21,131 | INFO : Options: check objectives and constraints.
2022-04-10 13:47:21,132 | INFO : Optimizer started.
2022-04-10 13:47:21,133 | INFO : Optimizer: build model...
2022-04-10 13:47:21,296 | INFO : Optimizer: solve...
2022-04-10 13:47:51,482 | INFO : Optimizer terminated. Time: 30.3493s
2022-04-10 13:47:51,483 | INFO : Post-processing started.
2022-04-10 13:47:51,513 | INFO : Post-processing terminated. Time: 0.0289s
2022-04-10 13:47:51,514 | INFO : Counterfactual generation terminated. Status: FEASIBLE. Time: 30.3959s


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,"[5.79, 6.82)",-,-,"[1.07, 1.10)","[986.50, 1426.50)",-,-,-,3.350462
0,-,-,-,-,"[986.50, 1426.50)","[3.11, 3.24)",-,"[-118.91, inf)",3.948499
0,"[5.79, 6.82)",-,-,-,"[986.50, 1426.50)","[2.90, 3.01)",-,-,3.233206


And the same generation enforcing diversity on feature values.

In [13]:
cf.generate(query=query, y=3.0, outcome_type="continuous", n_cf=3,
            max_changes=3,
            hard_constraints=["diversity_features", "diversity_values", "max_outcome"],
            time_limit=30
           ).display(show_only_changes=True, show_outcome=True)

2022-04-10 13:47:51,562 | INFO : Counterfactual generation started.
2022-04-10 13:47:51,564 | INFO : Options: check parameters.
2022-04-10 13:47:51,574 | INFO : Options: check objectives and constraints.
2022-04-10 13:47:51,576 | INFO : Optimizer started.
2022-04-10 13:47:51,578 | INFO : Optimizer: build model...
2022-04-10 13:47:51,710 | INFO : Optimizer: solve...
2022-04-10 13:48:22,120 | INFO : Optimizer terminated. Time: 30.5423s
2022-04-10 13:48:22,121 | INFO : Post-processing started.
2022-04-10 13:48:22,150 | INFO : Post-processing terminated. Time: 0.0273s
2022-04-10 13:48:22,150 | INFO : Counterfactual generation terminated. Status: FEASIBLE. Time: 30.5880s


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,"[5.79, 6.82)",-,-,-,"[986.50, 1426.50)","[3.52, 3.82)",-,-,2.918294
0,"[4.53, 5.04)",-,-,"[1.07, 1.10)","[1911.50, 2720.50)",-,-,-,2.620019
0,"[5.04, 5.79)",-,-,-,"[1426.50, 1911.50)",-,-,"[-118.91, inf)",2.744065
